### Regression table (Table 1)
- Load pre-fit regression artifacts.
- Print a readable coefficient table (coef [low, high]) per model.
- Save the LaTeX regression table to `../outputs/tables/regression_results.tex`.


In [1]:

from pathlib import Path
import importlib
import sys

# Ensure local src/ is importable when run from the notebook
THIS_DIR = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
SRC_DIR = THIS_DIR if (THIS_DIR / 'figure_two_regression.py').exists() else THIS_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

import figure_two_regression
importlib.reload(figure_two_regression)
from figure_two_regression import load_regression_artifacts, save_latex

RESULTS_PATH = Path('../outputs/models/regression_results.pkl')
TABLE_OUT_PATH = Path('../outputs/tables/regression_results.tex')

artifacts = load_regression_artifacts(RESULTS_PATH)
coef_df = artifacts['coef_df']
latex_str = artifacts['latex_str']


In [3]:

import pandas as pd

# Display options to avoid truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Build readable coefficient table: coef [low, high]
coef_df['coef_ci'] = coef_df.apply(lambda r: f"{r.coef:.3f} [{r.ci_low:.3f}, {r.ci_high:.3f}]", axis=1)
wide = coef_df.pivot_table(
    index='variable',
    columns=['estimator', 'model'],
    values='coef_ci',
    aggfunc='first',
    observed=False,
)
# Order columns: OLS first, then GLM, and models sorted alphabetically within estimator
wide = wide.reindex(columns=sorted(wide.columns, key=lambda x: (0 if x[0]=='OLS' else 1, x[1])))
print(wide.to_string())

# Metrics table
metrics_df = artifacts.get('metrics_df')
if metrics_df is not None:
    print("Model metrics (OLS first):")
    metrics_df = metrics_df.loc[sorted(metrics_df.index, key=lambda x: (0 if x.startswith('OLS') else 1, x))]
    print(metrics_df.to_string(float_format=lambda v: f"{v:.3f}" if pd.notnull(v) else ''))

# Save LaTeX table
save_latex(latex_str, TABLE_OUT_PATH)


estimator                              OLS                                                                        GLM                                                  
model                              OLS (1)                  OLS (2)                  OLS (3)                  GLM (1)                  GLM (2)                  GLM (3)
variable                                                                                                                                                               
C(MainPanel)[T.B]  -0.274 [-0.296, -0.252]  -0.275 [-0.297, -0.253]  -0.212 [-0.237, -0.186]  -1.330 [-1.431, -1.229]  -1.335 [-1.436, -1.234]  -1.067 [-1.183, -0.951]
C(MainPanel)[T.C]  -0.038 [-0.058, -0.017]  -0.041 [-0.061, -0.020]  -0.065 [-0.088, -0.042]  -0.154 [-0.233, -0.074]  -0.166 [-0.247, -0.086]  -0.293 [-0.388, -0.198]
C(MainPanel)[T.D]    0.009 [-0.016, 0.033]    0.007 [-0.018, 0.031]  -0.034 [-0.063, -0.004]    0.035 [-0.060, 0.129]    0.026 [-0.069, 0.121]  -0.170 [-0.293, 